# Overview

Our group chose to use data from two different streaming services (Netflix and Disney+) to create database that compares the different streaming services to a "master" movie list. 

https://www.kaggle.com/shivamb/netflix-shows
https://www.kaggle.com/shivamb/disney-movies-and-tv-shows

# Extract

We imported our libraries and extracted the CSV data from Kaggle for each streaming service

In [ ]:
#import libraries
import pandas as pd
from sqlalchemy import create_engine  

In [ ]:
#load CSV file for Dinsey+ data from Kaggle
csv_file = "Resources/disney_plus_titles.csv"
disney_df = pd.read_csv(csv_file)
disney_df.head()

In [ ]:
#load CSV file for Netflix data from Kaggle
csv_file = "Resources/netflix_titles.csv"
netflix_df = pd.read_csv(csv_file)
netflix_df.head()

# Transform

These CSV files have movie and TV show data in them. We filtered the movie data for each streaming service.

In [ ]:
disney_movies_df = disney_df.loc[disney_df['type'] == 'Movie']
disney_movies_df.count()

In [ ]:
netflix_movies_df = netflix_df.loc[netflix_df['type'] == 'Movie']
netflix_movies_df.count()

We created a streaming data dataframe to use in identifying which movies came from which streaming service later in the process.

In [ ]:
streaming_data = [[1, 'Netflix'], [2, 'Disney Plus']]
streaming_df = pd.DataFrame(streaming_data, columns = ['streaming_id', 'streaming_service_name'])
streaming_df

We combined the filtered Disney+ and Netflix dataframe into a new combined movie dataframe. We removed duplicate movies and added a movie_id index. This is our "master" movie list that is compared against with the individual lists later in the notebook. 

In [ ]:
#combined dataframe with all movies
combined_movies_df = pd.concat([disney_movies_df, netflix_movies_df], axis=0)
combined_movies_df.count()

In [ ]:
#remove duplicate movies from dataframe by title column
combined_movies_df = combined_movies_df.drop_duplicates(subset=['title'])
combined_movies_df

In [ ]:
#add movie ID index
combined_movies_df = combined_movies_df.assign(movie_id=combined_movies_df.reset_index().index + 1)
combined_movies_df

We renamed a column header from cast to cast_name because cast is a command in the SQL server. We dropped and reorganized the columns to match the database order.

In [ ]:
#change cast column header to cast_name for database
combined_movies_df.rename(columns={"cast": "cast_name"}, inplace=True)

In [ ]:
#reorganize and drop columns for database
combined_movies_df = combined_movies_df[["movie_id", 
                                         "title", 
                                         "director", 
                                         "cast_name", 
                                         "country", 
                                         "date_added", 
                                         "release_year", 
                                         "rating", 
                                         "duration", 
                                         "listed_in", 
                                         "description"]]
combined_movies_df

This section builds assoication tables that compare the "master" movie list and the individual streaming service lists. We did this by doing a joined for loop that creates a tuple of the movie id and streaming service number from the streaming service dataframe.

In [ ]:
# loop through the disney df, compare the title with the title in the combined_movie_df, build a tuple 

joined = []

for i, disney_row in disney_movies_df.iterrows():
    title = disney_row['title']
    for j, movie_row in combined_movies_df.iterrows():
        join = []
        if title == movie_row['title']:
            id = movie_row['movie_id']
            join = [id,2]
            joined.append(join)
            break
        
print(joined)

In [ ]:
disney_temp_df = pd.DataFrame(joined, columns = ['movie_id', 'streaming_id'])
disney_temp_df

In [ ]:
# loop through the netflix df, compare the title with the title in the combined_movie_df, build a tuple 

joined = []

for i, netflix_row in netflix_movies_df.iterrows():
    title = netflix_row['title']
    for j, movie_row in combined_movies_df.iterrows():
        join = []
        if title == movie_row['title']:
            id = movie_row['movie_id']
            join = [id,1]
            joined.append(join)
            break
        
print(joined)

In [ ]:
netflix_temp_df = pd.DataFrame(joined, columns = ['movie_id', 'streaming_id'])
netflix_temp_df

In [ ]:
#dataframe created to combine the results of for the join for loops of each streaming serice and the "master" movie list
movie_streaming_df = pd.concat([disney_temp_df, netflix_temp_df])
movie_streaming_df

# Load

This section we created the SQL database with three tables. Movies table that is the "master" movie list. Streaming which is the simple streaming service dataframe. Movie Streaming which is the combined dataframes from the joined for loop with which movies from the "master" movie list are on each streaming service.

In [ ]:
## BEFORE RUNNING THIS MAKE SURE YOU HAVE A LOCAL POSTGRES DB CREATED CALLED movies_db
## UPDATE YOUR CONNECTION STRING IF NEEDED 

rds_connection_string = "postgres:bootcamp@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
streaming_df.to_sql(name='streaming', con=engine, if_exists='append', index=False)

In [ ]:

combined_movies_df.to_sql(name='movies', con=engine, if_exists='append', index=False)

In [ ]:
# THIS DOESN"T WORK YET
movie_streaming_df.to_sql(name='movie_streaming', con=engine, if_exists='append', index=False)